# Workshop 03

- Name: Ran Arino
- Student ID: 153073200
- Email: rarino@myseneca.ca
- Course: Social Media Analytics
- Course ID: BDA600NAA.07578.2241
- Professor: Dr. Pantea Koochemeshkian

In [169]:
import pandas as pd
import numpy as np
import re

import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# read the data set from data folder
data = pd.read_csv("data/full-corpus-training.csv")
data.head()

,Sentiment,TweetId,TweetText
0,positive,1.260000e+17,Now all @Apple has to do is get swype on the i...
1,positive,1.260000e+17,@Apple will be adding more carrier support to ...
2,positive,1.260000e+17,Hilarious @youtube video - guy does a duet wit...
3,positive,1.260000e+17,@RIM you made it too easy for me to switch to ...
4,positive,1.260000e+17,I just realized that the reason I got into twi...


In [3]:
# remove the rows whose "Sentiment" column is "irrelevant"
new_data = data[data['Sentiment'] != 'irrelevant']
new_data['Sentiment'].value_counts()

Sentiment
neutral     2228
negative     437
positive     329
Name: count, dtype: int64

In [46]:
# cleaning the texts
def clean_texts(raw_texts: list or np.array):
    # define result
    result = []

    # set of stopwords
    stop_words = set(stopwords.words('english'))
    # set the porter stemming
    porter = nltk.PorterStemmer()
    # initialize tweet tokenizer
    tweet_tokenizer = TweetTokenizer()

    # function to convert emojis to text
    def convert_emojis(text):
        return emoji.demojize(text, delimiters=("", ""))

    # traversing all sentences
    for sent in raw_texts:
        # (1): white space removal
        sent = sent.strip()
        # (2): URL removal
        sent = re.sub(r"http[s]?://[\w?\W?]+", '', sent)
        # (3): HTML tag removal
        sent = re.sub(r'<[^>]+>', '', sent)
        # (4): Repeated words (at least four times)
        sent = re.sub(r'(.)\1{4,}', r'\1', sent)
        # (5): split attached words (at least two characters and follow the capitalized word)
        sent = re.sub(r"([\w]{2,})([A-Z])", r"\1 \2", sent)
        # (6): Punctuation removal without @ and #
        sent = re.sub(r'[^\w\s]', '', sent)
        # (7): Emoji to text
        sent = convert_emojis(sent)
        # (8): toknize exclude punct
        #tokens = nltk.regexp_tokenize(sent, r"\w+(?:'\w+)?")
        #(8): tokenize with TweetTokenizer
        tokens = tweet_tokenizer.tokenize(sent)

        # defined cleaned sentence
        clean_sent = ''
    
        # cleaning each sentence
        for w in tokens:
            # if 'w' is one of stop words, skip to the next word
            if w.lower() in stop_words:
                continue
            # add words without stemming
            clean_sent += porter.stem(w.lower()) + ' '

        # add clean_sent to result (make sure that the last item is always blank)
        result += [clean_sent[:-1]]

    return result

# get the clean tweet data as list
sent_list = clean_texts(np.array(new_data['TweetText'].values))
sent_list[:5]

['appl get swype iphon crack iphon',
 'appl ad carrier support iphon 4s announc',
 'hilari youtub video guy duet appl siri pretti much sum love affair',
 'ri made easi switch appl iphon see ya',
 'realiz reason got twitter io 5 thank appl']

In [53]:
# add clean text to the dataset
new_data.loc[:, 'CleanText'] = sent_list
new_data.head()

,Sentiment,TweetId,TweetText,CleanText
0,positive,1.260000e+17,Now all @Apple has to do is get swype on the i...,appl get swype iphon crack iphon
1,positive,1.260000e+17,@Apple will be adding more carrier support to ...,appl ad carrier support iphon 4s announc
2,positive,1.260000e+17,Hilarious @youtube video - guy does a duet wit...,hilari youtub video guy duet appl siri pretti ...
3,positive,1.260000e+17,@RIM you made it too easy for me to switch to ...,ri made easi switch appl iphon see ya
4,positive,1.260000e+17,I just realized that the reason I got into twi...,realiz reason got twitter io 5 thank appl


In [58]:
new_data['Sentiment'].value_counts()

Sentiment
neutral     2228
negative     437
positive     329
Name: count, dtype: int64

In [107]:
# apply tf-idf
# create TF-IDF vectrizer
tfidf_vect = TfidfVectorizer()
# fit and transform data
matrix = tfidf_vect.fit_transform(sent_list)

# get the explanatory and target variables for machine learning
X = matrix.toarray()
y = np.array(new_data['Sentiment'])
# Encord the target varieble for nueral network
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

In [109]:
print("Explnatory Variable format: ")
print(X[:5])

print("\nTarget Variable format: ")
print(y[:5])

print("\nEncorded Target Variable: ")
print(label_encoder.classes_)
print(y_cat[:5])

Explnatory Variable format: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Target Variable format: 
['positive' 'positive' 'positive' 'positive' 'positive']

Encorded Target Variable: 
['negative' 'neutral' 'positive']
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [172]:
# define the encoder for neural nerwork
label_encoder = LabelEncoder()
label_encoder.fit(y)

# define statifired k-fold processes
def strat_kfold(model_name: str, k=5):
    # stratified k-fold cross validation
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    # fold number
    i = 1
    # performance func
    perf_func = {"Acc": accuracy_score, "Pre": precision_score, "Rec": recall_score, 
                 "F1": f1_score, "Conf": confusion_matrix}
    # set the dict of the performance results
    results = {key: [] for key in perf_func.keys()}
    # trained model at each validation
    models = []
    # apply k-hold cross validation
    for train_index, test_index in skf.split(X, y):
        print(f"start fold {i}")
        # set train data
        X_train, X_test = X[train_index], X[test_index]
        # Neural Network
        if model_name == 'neural_network':
            # set test data
            y_train, y_test = y_cat[train_index], y_cat[test_index]
            # Neural Network Model
            model = Sequential()
            model.add(Dense(128, activation='relu', input_shape=(X.shape[1],)))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(3, activation='softmax'))  # 3 units for 3 classes
            # Compile the model
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            # Train the model
            model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)
            # predict the target classes
            y_pred = np.argmax(model.predict(X_test), axis=1)
            # convert the y_test to
            y_test = np.argmax(y_test, axis=1)
            # add model
            models.append(model)
        # random forest
        elif model_name == 'random_forest':
            # set the test data
            y_train, y_test = y[train_index], y[test_index]
            # set model
            model = RandomForestClassifier(
                n_estimators=100, class_weight='balanced',
                random_state=42, n_jobs=-1
                )
            # train the data into the model
            model.fit(X_train, y_train)
            # predict the target classes
            y_pred = model.predict(X_test)
            # add model
            models.append(model)
        # naive bayes
        elif model_name == "naive_bayes":
            # set the test data
            y_train, y_test = y[train_index], y[test_index]
            # set model
            model = MultinomialNB()
            # train the data into the model
            model.fit(X_train, y_train)
            # predict the target classes
            y_pred = model.predict(X_test)
            # add model
            models.append(model)

        else:
            return None, None, None

        # calculate classificaiton performances
        for key in perf_func.keys():
            if key in ["Acc", "Conf"]:
                params = {}
            else:
                params = {'average': 'macro'}
                
            results[key].append(perf_func[key](y_test, y_pred, **params))
        
        # increment fold
        i += 1

    # create the min-max confusion matrix
    stacked_arrays = np.stack(results['Conf'], axis=0)
    # Combine min and max values into a tuple for each component
    min_max_conf = np.dstack((np.min(stacked_arrays, axis=0), np.max(stacked_arrays, axis=0)))

    # delete "Conf" key from dict
    del results['Conf']

    return pd.DataFrame(results), min_max_conf, models


In [161]:
# Naive Bayes Classifiers
nb_results, nb_conf, nb_models = strat_kfold("naive_bayes")
print(nb_conf)
nb_results

start fold 1


C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


start fold 2
start fold 3


C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


start fold 4
start fold 5
[[[  1   2]
  [ 86  87]
  [  0   0]]

 [[  0   1]
  [445 446]
  [  0   0]]

 [[  0   0]
  [ 65  66]
  [  0   0]]]


C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Acc,Pre,Rec,F1
0,0.746244,0.581798,0.340909,0.299524
1,0.746244,0.581940,0.337165,0.292378
2,0.746244,0.581940,0.337165,0.292378
3,0.744574,0.415131,0.336417,0.291927
4,0.745819,0.581798,0.337121,0.292200


In [162]:
# Randon forest
rf_results, rf_conf, rf_models = strat_kfold("random_forest")
print(rf_conf)
rf_results

start fold 1
start fold 2
start fold 3
start fold 4
start fold 5
[[[ 28  37]
  [ 50  58]
  [  0   2]]

 [[  5  15]
  [425 434]
  [  2  12]]

 [[  0   0]
  [ 56  63]
  [  3  10]]]


,Acc,Pre,Rec,F1
0,0.779633,0.717071,0.453882,0.474983
1,0.784641,0.750272,0.470829,0.506274
2,0.796327,0.731607,0.513643,0.557706
3,0.771285,0.656357,0.470373,0.503527
4,0.779264,0.672582,0.473921,0.511021


In [173]:
# neural network
nw_results, nw_conf, nw_models = strat_kfold("neural_network")
print(nw_conf)
nw_results

start fold 1
Epoch 1/10
120/120 [==============================] - 4s 16ms/step - loss: 1.4880 - accuracy: 0.6879 - val_loss: 0.6282 - val_accuracy: 0.9562
Epoch 2/10
120/120 [==============================] - 2s 13ms/step - loss: 0.9075 - accuracy: 0.6915 - val_loss: 0.6379 - val_accuracy: 0.9562
Epoch 3/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8635 - accuracy: 0.6915 - val_loss: 0.6494 - val_accuracy: 0.9562
Epoch 4/10
120/120 [==============================] - 1s 13ms/step - loss: 0.8598 - accuracy: 0.6915 - val_loss: 0.7916 - val_accuracy: 0.9562
Epoch 5/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8583 - accuracy: 0.6921 - val_loss: 0.7613 - val_accuracy: 0.9562
Epoch 6/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8557 - accuracy: 0.6952 - val_loss: 0.8489 - val_accuracy: 0.9332
Epoch 7/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8517 - accuracy: 0.6983 - val_loss: 0.8775 - val_acc

C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 3s 16ms/step - loss: 1.5130 - accuracy: 0.6597 - val_loss: 0.6291 - val_accuracy: 0.9749
Epoch 2/10
120/120 [==============================] - 2s 14ms/step - loss: 0.9060 - accuracy: 0.6863 - val_loss: 0.5537 - val_accuracy: 0.9749
Epoch 3/10
120/120 [==============================] - 2s 14ms/step - loss: 0.8765 - accuracy: 0.6863 - val_loss: 0.5548 - val_accuracy: 0.9749
Epoch 4/10
120/120 [==============================] - 2s 15ms/step - loss: 0.8683 - accuracy: 0.6863 - val_loss: 0.5547 - val_accuracy: 0.9749
Epoch 5/10
120/120 [==============================] - 2s 14ms/step - loss: 0.8653 - accuracy: 0.6863 - val_loss: 0.5820 - val_accuracy: 0.9749
Epoch 6/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8656 - accuracy: 0.6863 - val_loss: 0.6066 - val_accuracy: 0.9749
Epoch 7/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8677 - accuracy: 0.6863 - val_loss: 0.5941 - val_accuracy: 0.9749

C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 6s 15ms/step - loss: 1.4904 - accuracy: 0.6874 - val_loss: 0.6652 - val_accuracy: 0.9374
Epoch 2/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8984 - accuracy: 0.6957 - val_loss: 0.5933 - val_accuracy: 0.9374
Epoch 3/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8617 - accuracy: 0.6957 - val_loss: 0.6200 - val_accuracy: 0.9374
Epoch 4/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8583 - accuracy: 0.6957 - val_loss: 0.6987 - val_accuracy: 0.9374
Epoch 5/10
120/120 [==============================] - 2s 14ms/step - loss: 0.8587 - accuracy: 0.6957 - val_loss: 0.6943 - val_accuracy: 0.9374
Epoch 6/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8575 - accuracy: 0.6957 - val_loss: 0.6960 - val_accuracy: 0.9374
Epoch 7/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8499 - accuracy: 0.6957 - val_loss: 0.7644 - val_accuracy: 0.9374

...

Epoch 1/10
120/120 [==============================] - 3s 15ms/step - loss: 1.4994 - accuracy: 0.6639 - val_loss: 0.6609 - val_accuracy: 0.9896
Epoch 2/10
120/120 [==============================] - 2s 13ms/step - loss: 0.9096 - accuracy: 0.6827 - val_loss: 0.5447 - val_accuracy: 0.9896
Epoch 3/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8742 - accuracy: 0.6827 - val_loss: 0.5365 - val_accuracy: 0.9896
Epoch 4/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8794 - accuracy: 0.6827 - val_loss: 0.5949 - val_accuracy: 0.9896
Epoch 5/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8735 - accuracy: 0.6827 - val_loss: 0.6496 - val_accuracy: 0.9896
Epoch 6/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8704 - accuracy: 0.6827 - val_loss: 0.7290 - val_accuracy: 0.9896
Epoch 7/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8629 - accuracy: 0.6827 - val_loss: 0.7371 - val_accuracy: 0.9896

C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
120/120 [==============================] - 3s 14ms/step - loss: 1.4794 - accuracy: 0.6910 - val_loss: 0.6525 - val_accuracy: 0.9333
Epoch 2/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8956 - accuracy: 0.6968 - val_loss: 0.6122 - val_accuracy: 0.9333
Epoch 3/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8636 - accuracy: 0.6968 - val_loss: 0.5881 - val_accuracy: 0.9333
Epoch 4/10
120/120 [==============================] - 1s 12ms/step - loss: 0.8592 - accuracy: 0.6968 - val_loss: 0.6828 - val_accuracy: 0.9333
Epoch 5/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8675 - accuracy: 0.6968 - val_loss: 0.7323 - val_accuracy: 0.9333
Epoch 6/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8601 - accuracy: 0.6973 - val_loss: 0.6876 - val_accuracy: 0.9333
Epoch 7/10
120/120 [==============================] - 2s 13ms/step - loss: 0.8527 - accuracy: 0.6978 - val_loss: 0.6939 - val_accuracy: 0.9333

C:\Users\runru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Acc,Pre,Rec,F1
0,0.729549,0.382523,0.403311,0.388660
1,0.744574,0.248191,0.333333,0.284530
2,0.732888,0.274788,0.331186,0.289612
3,0.744574,0.248191,0.333333,0.284530
4,0.744147,0.399046,0.381954,0.369628


In [164]:
import numpy as np

# Averaging weights of the models
weights = [model.get_weights() for model in nw_models]
new_weights = list()

for weights_list_tuple in zip(*weights):
    new_weights.append(
        np.array([np.array(weights_).mean(axis=0) for weights_ in zip(*weights_list_tuple)])
    )

# Create a new model with the same architecture
aggregated_model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=l2(0.01)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(3, activation='softmax')
])

# Set the averaged weights
aggregated_model.set_weights(new_weights)


### Load Test Data

In [165]:
# check the apply test data
testing = pd.read_excel('data/testing_data.xlsx', header=None)
testing = testing.rename(columns={i: col for i, col in enumerate(['TweetId', "TweetText"])})
testing.head()

,TweetId,TweetText
0,126352268705538000,Come to the dark side üì±‚Äú@gretcheneclark: ...
1,126350302113824000,"Hey @apple, if you send me a free iPhone (any ..."
2,126349695676203008,Thank you @apple for Find My Mac - just locate...
3,126342268603998000,Thanks to @Apple Covent Garden #GeniusBar for ...
4,126325800080392000,@DailyDealChat @apple Thanks!!


In [166]:
# get the clean tweet data as list
sent_list_test = clean_texts(np.array(testing['TweetText'].values))
# transform to the TF-IDF
matrix_test = tfidf_vect.transform(sent_list_test)
test_data = matrix_test.toarray()
test_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [167]:
# show the predicted labels
test_data_pred = aggregated_model.predict(test_data)
# convert to the label and invert the encoded labels
test_data_pred_cat = label_encoder.inverse_transform(np.argmax(test_data_pred, axis=1))


15/15 [==============================] - 0s 5ms/step


In [168]:
sum(testing_answer[1].values == test_data_pred_cat)

104